<a href="https://colab.research.google.com/github/Tapish0305/Stock_price_prediction/blob/main/notebook_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTING DEPENDENCIES

In [ ]:
!pip install yahoo_fin

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.8/125.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 9.0 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=33925312934e248bb206626f2fe66f07b996eb43adac9dac814f2394af0dd22e
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling websockets-15.0.1:
      Successfully uninstalled websockets-15.0.1
  Attempting uninst

In [ ]:
!pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=5476188490e27a49f0850b046bf255df3e269c0768cc30e711414a196e002d23
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.preprocessing import MinMaxScaler
from yahoo_fin import news
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import GRU, Dense, Dropout, LSTM
from tensorflow.keras.losses import MeanSquaredError

import torch.nn.utils.rnn as rnn
import torch.nn.functional as F
import torch.nn as nn

TESLA AND APPLE STOCK PRICE NEWS DATA

In [ ]:
data = pd.read_csv('/content/combined.csv')
index_values0 = data.loc[data['ticker'] == 'AAPL']
index_values1 = data.loc[data['ticker'] == 'TSLA']

In [ ]:
index_values0.head()

,ticker,headline,date,eventRet,Close
22,AAPL,Despite a wave of analyst love regarding yeste...,2011-06-07,-0.032871,10.093049
23,AAPL,Bill Ackman's hedge fund made ~$490M in one da...,2011-06-14,0.002913,10.105204
28,AAPL,Apple (AAPL -1.5%) shares continue their perpl...,2011-06-20,0.001156,9.584808
47,AAPL,"After lagging for months, tech bellwethers App...",2011-07-07,0.029160,10.857841
57,AAPL,"Looking at a ""refresh of virtually every produ...",2011-07-13,0.010666,10.882765


In [ ]:
index_values1.head()

,ticker,headline,date,eventRet,Close
18,TSLA,Tesla Motors (TSLA +3.6%) announces a follow-o...,2011-05-25,0.098105,1.932000
21,TSLA,Tesla Motors (TSLA +6.3%) jumps premarket afte...,2011-06-03,0.008590,2.008667
71,TSLA,Tesla Motors (TSLA +4.5%) scores a much bigger...,2011-07-20,0.053271,1.912667
103,TSLA,A video rumored to be released back-channels b...,2011-08-12,0.099013,1.754000
141,TSLA,Tesla Motors (TSLA +1.5%) gains on positive co...,2011-09-21,-0.005349,1.723333


In [ ]:
news0 = index_values0[['headline','date','eventRet']]
news1 = index_values1[['headline','date','eventRet']]

In [ ]:
news0['Date'] = news0['date']
news1['Date'] = news1['date']

<ipython-input-8-4546fdcd0dd7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news0['Date'] = news0['date']
<ipython-input-8-4546fdcd0dd7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news1['Date'] = news1['date']


In [ ]:
news0 = news0.drop('date', axis = 1)
news1 = news1.drop('date', axis = 1)

In [ ]:
news0.head()

,headline,eventRet,Date
22,Despite a wave of analyst love regarding yeste...,-0.032871,2011-06-07
23,Bill Ackman's hedge fund made ~$490M in one da...,0.002913,2011-06-14
28,Apple (AAPL -1.5%) shares continue their perpl...,0.001156,2011-06-20
47,"After lagging for months, tech bellwethers App...",0.029160,2011-07-07
57,"Looking at a ""refresh of virtually every produ...",0.010666,2011-07-13


In [ ]:
news1.head()

,headline,eventRet,Date
18,Tesla Motors (TSLA +3.6%) announces a follow-o...,0.098105,2011-05-25
21,Tesla Motors (TSLA +6.3%) jumps premarket afte...,0.008590,2011-06-03
71,Tesla Motors (TSLA +4.5%) scores a much bigger...,0.053271,2011-07-20
103,A video rumored to be released back-channels b...,0.099013,2011-08-12
141,Tesla Motors (TSLA +1.5%) gains on positive co...,-0.005349,2011-09-21


TESLA AND APPLE STOCK PRICE DATA

In [ ]:
# Define top companies (e.g., Apple, Microsoft, Amazon, Google, Nvidia)
top_companies0 = [ 'AAPL']
top_companies1 = ['TSLA']
# Download historical stock data
stock_data0 = yf.download(top_companies0, start="2011-05-02", end="2018-12-24", interval="1d")
stock_data1 = yf.download(top_companies1, start="2011-05-02", end="2018-12-24", interval="1d")
# Save as CSV
stock_data0.to_csv("top_sp500_stock_data.csv")
stock_data1.to_csv("top_sp500_stock_data.csv")

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
stock_data0 = stock_data0.droplevel(1, axis=1)
stock_data1 = stock_data1.droplevel(1, axis=1)

In [ ]:
stock_data0 = stock_data0.reset_index()
stock_data1 = stock_data1.reset_index()

In [ ]:
stock_data0.head()

Price,Date,Close,High,Low,Open,Volume
0,2011-05-02,10.420808,10.546900,10.397335,10.524932,442713600
1,2011-05-03,10.478591,10.529449,10.400949,10.472272,313348000
2,2011-05-04,10.519819,10.587831,10.438867,10.480396,389250400
3,2011-05-05,10.434958,10.561351,10.413892,10.484612,335969200
4,2011-05-06,10.432249,10.532762,10.418707,10.523433,280134400


In [ ]:
stock_data1.head()

Price,Date,Close,High,Low,Open,Volume
0,2011-05-02,1.830000,1.853333,1.804000,1.840000,11769000
1,2011-05-03,1.791333,1.826000,1.766667,1.825333,13708500
2,2011-05-04,1.779333,1.800000,1.716667,1.785333,15667500
3,2011-05-05,1.762667,1.829333,1.744667,1.813333,18277500
4,2011-05-06,1.808000,1.846667,1.774667,1.793333,14725500


CONCATENATING NEWS AND STOCK PRICE DATA ACCORDING TO DATE

In [ ]:
import pandas as pd

# Ensure both DataFrames have "Date" in datetime format
stock_data0["Date"] = pd.to_datetime(stock_data0["Date"], errors="coerce")
news0["Date"] = pd.to_datetime(news0["Date"], errors="coerce")  # Fix column name if needed

# Merge the DataFrames on 'Date', keeping only common dates
merged_df0 = pd.merge(stock_data0, news0, on="Date", how="inner")

# Sort by Date and reset index
merged_df0 = merged_df0.sort_values(by="Date").reset_index(drop=True)

merged_df0.head()

,Date,Close,High,Low,Open,Volume,headline,eventRet
0,2011-06-07,9.992280,10.178259,9.988067,10.176754,529785200,Despite a wave of analyst love regarding yeste...,-0.032871
1,2011-06-14,10.004317,10.028693,9.910124,9.930889,334569200,Bill Ackman's hedge fund made ~$490M in one da...,0.002913
2,2011-06-20,9.489116,9.560739,9.344064,9.550506,640645600,Apple (AAPL -1.5%) shares continue their perpl...,0.001156
3,2011-07-07,10.749433,10.773507,10.653132,10.673296,399663600,"After lagging for months, tech bellwethers App...",0.029160
4,2011-07-13,10.774109,10.833694,10.724754,10.783437,391638800,"Looking at a ""refresh of virtually every produ...",0.010666


In [ ]:
import pandas as pd

# Ensure both DataFrames have "Date" in datetime format
stock_data1["Date"] = pd.to_datetime(stock_data1["Date"], errors="coerce")
news1["Date"] = pd.to_datetime(news1["Date"], errors="coerce")  # Fix column name if needed

# Merge the DataFrames on 'Date', keeping only common dates
merged_df1 = pd.merge(stock_data1, news1, on="Date", how="inner")

# Sort by Date and reset index
merged_df1 = merged_df1.sort_values(by="Date").reset_index(drop=True)

merged_df1.head()

,Date,Close,High,Low,Open,Volume,headline,eventRet
0,2011-05-25,1.932000,1.934000,1.744667,1.793333,70396500,Tesla Motors (TSLA +3.6%) announces a follow-o...,0.098105
1,2011-06-03,2.008667,2.100000,1.966667,1.996667,93138000,Tesla Motors (TSLA +6.3%) jumps premarket afte...,0.008590
2,2011-07-20,1.912667,2.029333,1.853333,1.866667,45724500,Tesla Motors (TSLA +4.5%) scores a much bigger...,0.053271
3,2011-08-12,1.754000,1.809333,1.690667,1.706667,15136500,A video rumored to be released back-channels b...,0.099013
4,2011-09-21,1.723333,1.796667,1.713333,1.730000,14814000,Tesla Motors (TSLA +1.5%) gains on positive co...,-0.005349


In [ ]:
merged_df0.dropna(inplace = True)
merged_df1.dropna(inplace = True)

In [ ]:
merged_df0.shape

(462, 8)

In [ ]:
merged_df1.shape

(479, 8)

ADDING NEW FEATURE

In [ ]:
merged_df0['Daily_Return'] = stock_data0['Close'].pct_change()
merged_df1['Daily_Return'] = stock_data1['Close'].pct_change()

In [ ]:
## EXPONENTIAL MOVING AVERAGE
merged_df0['EMA_100'] = merged_df0['Close'].ewm(span=10, adjust=False).mean()
merged_df1['EMA_100'] = merged_df1['Close'].ewm(span=10, adjust=False).mean()
## BOLLINGER DEMANDS
import ta
merged_df0['RSI'] = ta.momentum.RSIIndicator(merged_df0['Close'], window=14).rsi()
merged_df1['RSI'] = ta.momentum.RSIIndicator(merged_df1['Close'], window=14).rsi()
## Volatility
merged_df0['Volatility_10'] = merged_df0['Daily_Return'].rolling(window=10).std()
merged_df1['Volatility_10'] = merged_df1['Daily_Return'].rolling(window=10).std()

In [ ]:
price0 = merged_df0.dropna()
price1 = merged_df1.dropna()

In [ ]:
price0.head()

,Date,Close,High,Low,Open,Volume,headline,eventRet,Daily_Return,EMA_100,RSI,Volatility_10
14,2011-08-25,11.246580,11.298642,10.984164,10.986571,871346000,"Let's get real: In the long run, Apple (AAPL -...",0.026750,-0.015594,11.172054,63.015431,0.011054
15,2011-08-25,11.246580,11.298642,10.984164,10.986571,871346000,Apple (NASDAQ:AAPL) is now down just 1.6% in t...,0.026750,-0.002446,11.185604,63.015431,0.010997
16,2011-08-26,11.543302,11.549922,11.158705,11.169839,641477200,Apple (AAPL +2.6%) shares rallied today to ful...,0.036503,-0.006608,11.250640,68.061727,0.010751
17,2011-09-19,12.387428,12.435578,11.892992,11.947159,823860800,Apple (AAPL +2%) is outperforming strongly tod...,0.051400,0.013817,11.457329,77.476831,0.012255
18,2011-09-26,12.132833,12.157208,11.775622,12.033222,812876400,"U.S. shares are higher across the board, with ...",-0.006321,-0.005286,11.580148,70.706623,0.012269


In [ ]:
price1.head()

,Date,Close,High,Low,Open,Volume,headline,eventRet,Daily_Return,EMA_100,RSI,Volatility_10
13,2012-03-22,2.293333,2.343333,2.286667,2.331333,7836000,Luxury electric auto maker Tesla Motors (TSLA ...,-0.025205,0.070209,1.935856,62.384754,0.034413
14,2012-03-26,2.493333,2.539333,2.336000,2.372667,47107500,Tesla Motors (TSLA +3.7%) is upgraded to Buy f...,0.102554,-0.008156,2.037215,66.732863,0.034030
15,2012-04-03,2.534000,2.564667,2.444667,2.446667,16471500,Shares of Tesla Motors (TSLA +2.9%) kick up a ...,-0.057820,-0.041116,2.127540,67.554142,0.035399
16,2012-04-09,2.210000,2.286000,2.206667,2.273333,24835500,Abu Dhabi National Energy sells off its stake ...,-0.074245,-0.003729,2.142532,55.746075,0.034803
17,2012-05-09,2.004000,2.051333,1.984000,2.020000,29218500,More on Tesla Motors (NASDAQ:TSLA): Q1 misses ...,0.021949,0.084581,2.117345,49.787353,0.041871


In [ ]:
price0['Volume'] = price0['Volume']/10e6
price1['Volume'] = price1['Volume']/10e6

<ipython-input-27-83dc1eb114d1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price0['Volume'] = price0['Volume']/10e6
<ipython-input-27-83dc1eb114d1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price1['Volume'] = price1['Volume']/10e6


Performing Roubust Scaling on data

In [ ]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
cs0 = scaler.fit_transform( price0[['Close','High','Low','Open','Volume','eventRet',
                                                                              'Daily_Return','EMA_100','RSI','Volatility_10']])
df0 = pd.DataFrame(cs0)

In [ ]:
cs1 = scaler.fit_transform( price1[['Close','High','Low','Open','Volume','eventRet',
                                                                              'Daily_Return','EMA_100','RSI','Volatility_10']])
df1 = pd.DataFrame(cs1)

Assigning Features Name

In [ ]:
df0.rename(columns={0:'Close',1:'High',2:'Low',3:'Open',
                   4: 'Volume',5:'eventRet',6:'Daily_Return',
                   7:'EMA_100', 8:'RSI', 9: 'Volatility_10'}, inplace=True)

In [ ]:
df1.rename(columns={0:'Close',1:'High',2:'Low',3:'Open',
                   4: 'Volume',5:'eventRet',6:'Daily_Return',
                   7:'EMA_100', 8:'RSI', 9: 'Volatility_10'}, inplace=True)

In [ ]:
price0 = price0.reset_index()
price1 = price1.reset_index()

In [ ]:
df0['headline']	= price0['headline']
df0['date'] = price0['Date']

In [ ]:
df1['headline']	= price1['headline']
df1['date'] = price1['Date']

In [ ]:
df0.head()

,Close,High,Low,Open,Volume,eventRet,Daily_Return,EMA_100,RSI,Volatility_10,headline,date
0,-0.632381,-0.638992,-0.635903,-0.640852,1.474185,0.498016,-0.732313,-0.619691,0.163417,-0.496502,"Let's get real: In the long run, Apple (AAPL -...",2011-08-25
1,-0.632381,-0.638992,-0.635903,-0.640852,1.474185,0.498016,-0.115556,-0.619120,0.163417,-0.502687,Apple (NASDAQ:AAPL) is now down just 1.6% in t...,2011-08-25
2,-0.619890,-0.628485,-0.628543,-0.633196,0.941926,0.682898,-0.310808,-0.616377,0.366423,-0.529209,Apple (AAPL +2.6%) shares rallied today to ful...,2011-08-26
3,-0.584355,-0.591454,-0.597581,-0.600724,1.364234,0.965288,0.647375,-0.607660,0.745180,-0.367132,Apple (AAPL +2%) is outperforming strongly tod...,2011-09-19
4,-0.595073,-0.603093,-0.602530,-0.597128,1.338799,-0.128882,-0.248752,-0.602480,0.472824,-0.365676,"U.S. shares are higher across the board, with ...",2011-09-26


In [ ]:
df1.head()

,Close,High,Low,Open,Volume,eventRet,Daily_Return,EMA_100,RSI,Volatility_10,headline,date
0,-1.575085,-1.639529,-1.560488,-1.582101,-1.005321,-0.232765,2.002755,-1.853741,0.384418,0.362720,Luxury electric auto maker Tesla Motors (TSLA ...,2012-03-22
1,-1.549470,-1.613812,-1.554022,-1.576701,-0.620771,1.140055,-0.232651,-1.838819,0.595292,0.340985,Tesla Motors (TSLA +3.7%) is upgraded to Buy f...,2012-03-26
2,-1.544261,-1.610488,-1.539779,-1.567033,-0.920761,-0.583233,-1.172859,-1.825523,0.635122,0.418674,Shares of Tesla Motors (TSLA +2.9%) kick up a ...,2012-04-03
3,-1.585758,-1.647051,-1.570973,-1.589679,-0.838860,-0.759717,-0.106366,-1.823316,0.062457,0.384876,Abu Dhabi National Energy sells off its stake ...,2012-04-09
4,-1.612142,-1.677841,-1.600157,-1.622776,-0.795942,0.273919,2.412745,-1.827024,-0.226528,0.786095,More on Tesla Motors (NASDAQ:TSLA): Q1 misses ...,2012-05-09


# Using Finbert to performing sentiment analysis on stock price news data

In [ ]:
!pip install feedparser

In [ ]:
import feedparser
import pandas as pd

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load FinBERT model (trained for financial sentiment analysis)
model_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
df0.head()

,Close,High,Low,Open,Volume,eventRet,Daily_Return,EMA_100,RSI,Volatility_10,headline,date
0,-0.632381,-0.638992,-0.635903,-0.640852,1.474185,0.498016,-0.732313,-0.619691,0.163417,-0.496502,"Let's get real: In the long run, Apple (AAPL -...",2011-08-25
1,-0.632381,-0.638992,-0.635903,-0.640852,1.474185,0.498016,-0.115556,-0.619120,0.163417,-0.502687,Apple (NASDAQ:AAPL) is now down just 1.6% in t...,2011-08-25
2,-0.619890,-0.628485,-0.628543,-0.633196,0.941926,0.682898,-0.310808,-0.616377,0.366423,-0.529209,Apple (AAPL +2.6%) shares rallied today to ful...,2011-08-26
3,-0.584355,-0.591454,-0.597581,-0.600724,1.364234,0.965288,0.647375,-0.607660,0.745180,-0.367132,Apple (AAPL +2%) is outperforming strongly tod...,2011-09-19
4,-0.595073,-0.603093,-0.602530,-0.597128,1.338799,-0.128882,-0.248752,-0.602480,0.472824,-0.365676,"U.S. shares are higher across the board, with ...",2011-09-26


In [ ]:
df1.head()

,Close,High,Low,Open,Volume,eventRet,Daily_Return,EMA_100,RSI,Volatility_10,headline,date
0,-1.575085,-1.639529,-1.560488,-1.582101,-1.005321,-0.232765,2.002755,-1.853741,0.384418,0.362720,Luxury electric auto maker Tesla Motors (TSLA ...,2012-03-22
1,-1.549470,-1.613812,-1.554022,-1.576701,-0.620771,1.140055,-0.232651,-1.838819,0.595292,0.340985,Tesla Motors (TSLA +3.7%) is upgraded to Buy f...,2012-03-26
2,-1.544261,-1.610488,-1.539779,-1.567033,-0.920761,-0.583233,-1.172859,-1.825523,0.635122,0.418674,Shares of Tesla Motors (TSLA +2.9%) kick up a ...,2012-04-03
3,-1.585758,-1.647051,-1.570973,-1.589679,-0.838860,-0.759717,-0.106366,-1.823316,0.062457,0.384876,Abu Dhabi National Energy sells off its stake ...,2012-04-09
4,-1.612142,-1.677841,-1.600157,-1.622776,-0.795942,0.273919,2.412745,-1.827024,-0.226528,0.786095,More on Tesla Motors (NASDAQ:TSLA): Q1 misses ...,2012-05-09


In [ ]:
def get_sentiment(text):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get label with highest probability
    probs = F.softmax(outputs.logits, dim=-1)

    # Convert label to sentiment
    return (probs[0][2]-probs[0][0]).numpy().item() ## positive - negative

In [ ]:
df0.columns

Index(['Close', 'High', 'Low', 'Open', 'Volume', 'eventRet', 'Daily_Return',
       'EMA_100', 'RSI', 'Volatility_10', 'headline', 'date'],
      dtype='object')

In [ ]:
df0['sentiment'] = df0['headline'].apply(get_sentiment)
df1['sentiment'] = df1['headline'].apply(get_sentiment)

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
df0 = df0.drop(['headline','date'], axis=1)
df1 = df1.drop(['headline','date'], axis=1)

In [ ]:
df0.head()

,Close,High,Low,Open,Volume,eventRet,Daily_Return,EMA_100,RSI,Volatility_10,sentiment
0,-0.632381,-0.638992,-0.635903,-0.640852,1.474185,0.498016,-0.732313,-0.619691,0.163417,-0.496502,0.415604
1,-0.632381,-0.638992,-0.635903,-0.640852,1.474185,0.498016,-0.115556,-0.619120,0.163417,-0.502687,0.008747
2,-0.619890,-0.628485,-0.628543,-0.633196,0.941926,0.682898,-0.310808,-0.616377,0.366423,-0.529209,-0.926992
3,-0.584355,-0.591454,-0.597581,-0.600724,1.364234,0.965288,0.647375,-0.607660,0.745180,-0.367132,-0.538386
4,-0.595073,-0.603093,-0.602530,-0.597128,1.338799,-0.128882,-0.248752,-0.602480,0.472824,-0.365676,-0.920180


In [ ]:
df1.head()

,Close,High,Low,Open,Volume,eventRet,Daily_Return,EMA_100,RSI,Volatility_10,sentiment
0,-1.575085,-1.639529,-1.560488,-1.582101,-1.005321,-0.232765,2.002755,-1.853741,0.384418,0.362720,0.015583
1,-1.549470,-1.613812,-1.554022,-1.576701,-0.620771,1.140055,-0.232651,-1.838819,0.595292,0.340985,-0.842692
2,-1.544261,-1.610488,-1.539779,-1.567033,-0.920761,-0.583233,-1.172859,-1.825523,0.635122,0.418674,-0.918571
3,-1.585758,-1.647051,-1.570973,-1.589679,-0.838860,-0.759717,-0.106366,-1.823316,0.062457,0.384876,0.536333
4,-1.612142,-1.677841,-1.600157,-1.622776,-0.795942,0.273919,2.412745,-1.827024,-0.226528,0.786095,-0.915175


In [ ]:
df0.shape

(449, 11)

In [ ]:
df1.shape

(466, 11)

In [ ]:
X_train0 = df0[:380]
X_valid0 = df0[380:]

In [ ]:
X_train0.shape

(380, 11)

In [ ]:
X_valid0.shape

(69, 11)

Creating time series sequence as input to GRU and LSTM

In [ ]:
def create_seq(df, seq_len):
  x_train = []
  y_train = []
  for i in range(len(df)-seq_len):
    x_train.append(df.iloc[i:(i+seq_len)])
    y_train.append(df.iloc[i+seq_len,0])
  return np.array(x_train), np.array(y_train)

In [ ]:
x_train0, y_train0 = create_seq(X_train0, seq_len = 60)
x_valid0, y_valid0 = create_seq(X_valid0, seq_len = 60)

In [ ]:
x1, y1 = create_seq(df1, seq_len = 60)

In [ ]:
!pip install transformers

# GRU MODEL

In [ ]:
model= tf.keras.Sequential([
    GRU(64, return_sequences=True, kernel_regularizer=l2(0.01)),  # L2 Regularization
    Dropout(0.2),  # 20% neurons ko random drop karenge
    Dense(1)  # Final output layer
])
model.compile(loss='mean_squared_error',optimizer='adam')

In [ ]:
model.fit(x_train0, y_train0, epochs = 25)

Epoch 1/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 0.4320
Epoch 2/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.3776
Epoch 3/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.3556
Epoch 4/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.3223
Epoch 5/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.3136
Epoch 6/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.2988
Epoch 7/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.2885
Epoch 8/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.2679
Epoch 9/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.2597
Epoch 10/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.2407
Epoch 11/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.2455
Epoch 12/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 0.2339
Epoch 13/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 0.2253
Epoch 14/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 0.2252
Epoch 15/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.2205
Epoc

In [ ]:
y_pred1 = model.predict(x1)
y_pred0 = model.predict(x_valid0)

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


In [ ]:
mse = tf.keras.losses.MeanSquaredError()

In [ ]:
mean_absolute_error0 = mse(y_pred0, y_valid0)
mean_absolute_error1 = mse(y_pred1, y1)

In [ ]:
mean_absolute_error0

<tf.Tensor: shape=(), dtype=float32, numpy=0.24716265499591827>

In [ ]:
mean_absolute_error1

<tf.Tensor: shape=(), dtype=float32, numpy=0.3544851839542389>

# LSTM MODEL

In [ ]:
model1= tf.keras.Sequential([
    LSTM(32, return_sequences=True, kernel_regularizer=l2(0.001)),  # L2 Regularization
    Dropout(0.2),
    Dense(1),
])
model1.compile(loss='mean_squared_error',optimizer='adam')

In [ ]:
model1.fit(x_train0, y_train0, epochs = 25)

Epoch 1/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - loss: 0.2495
Epoch 2/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.2397
Epoch 3/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.2222
Epoch 4/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.2244
Epoch 5/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.2114
Epoch 6/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.2199
Epoch 7/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.2137
Epoch 8/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.2136
Epoch 9/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.2133
Epoch 10/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.2104
Epoch 11/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.2021
Epoch 12/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.2043
Epoch 13/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.2126
Epoch 14/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.2068
Epoch 15/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.2030
Epoc

In [ ]:
y_pred1 = model1.predict(x1)
y_pred0 = model1.predict(x_valid0)

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


In [ ]:
mean_absolute_error0 = mse(y_pred0, y_valid0)
mean_absolute_error1 = mse(y_pred1, y1)

In [ ]:
mean_absolute_error0

<tf.Tensor: shape=(), dtype=float32, numpy=0.30025941133499146>

In [ ]:
mean_absolute_error1

<tf.Tensor: shape=(), dtype=float32, numpy=0.3500560522079468>

# ADVERSINAL TRAINING

In [ ]:
# model2 = tf.keras.Sequential([
#      GRU(32, return_sequences=True, kernel_regularizer=l2(0.001)),  # L2 Regularization
#      Dropout(0.3),  # 30% neurons ko random drop karenge
#      GRU(16, return_sequences=False, kernel_regularizer=l2(0.001)),
#      Dropout(0.3),
#      Dense(1)  # Final output layer
#  ])

In [ ]:
# model2.compile(loss='mean_squared_error',optimizer='adam')

In [ ]:
# mse = MeanSquaredError()
# def create_adversarial_pattern(model, x, y_true, epsilon=0.01, alpha=0.001, num_steps=7):
#     x = tf.convert_to_tensor(x, dtype=tf.float32)
#     y_true = tf.convert_to_tensor(y_true, dtype=tf.float32)
#      # Initialize adversarial example with the original input
#     adversarial_x = tf.identity(x)
#     for _ in range(num_steps):
#         with tf.GradientTape() as tape:
#             tape.watch(adversarial_x)
#             y_pred = model(adversarial_x, training=True)
#             loss = mse(y_true, y_pred)

#         # Compute the gradient
#         gradient = tape.gradient(loss, adversarial_x)
#         signed_grad = tf.sign(gradient)  # Get sign of the gradient

#          # Apply small perturbation step
#         adversarial_x = adversarial_x + alpha * signed_grad

# #         # Project the adversarial example to ensure it stays in the epsilon-ball
#         perturbation = tf.clip_by_value(adversarial_x - x, -epsilon, epsilon)
#         adversarial_x = tf.clip_by_value(x + perturbation, 0, 1)  # Ensure valid input range

#     return adversarial_x.numpy()

In [ ]:
# x_train_adv = create_adversarial_pattern(model2, x_train0, y_train0)

In [ ]:
# x_valid_adv = create_adversarial_pattern(model2, x_valid0, y_valid0)

In [ ]:
# x_train_adv.shape

(320, 60, 11)

In [ ]:
# x_train_combined = np.vstack([x_train0, x_train_adv])
# y_train_combined = np.hstack([y_train0, y_train0])

In [ ]:
# x_valid_combined = np.vstack([x_valid0, x_valid_adv])
# y_valid_combined = np.hstack([y_valid0, y_valid0])

In [ ]:
# model2.fit(x_train_combined, y_train_combined, validation_data =(x_valid_combined, y_valid_combined),  epochs = 17)

Epoch 1/17
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 112ms/step - loss: 0.0382 - val_loss: 0.0309
Epoch 2/17
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - loss: 0.0388 - val_loss: 0.0214
Epoch 3/17
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step - loss: 0.0350 - val_loss: 0.0225
Epoch 4/17
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - loss: 0.0346 - val_loss: 0.0218
Epoch 5/17
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 0.0343 - val_loss: 0.0192
Epoch 6/17
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 94ms/step - loss: 0.0322 - val_loss: 0.0204
Epoch 7/17
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - loss: 0.0302 - val_loss: 0.0226
Epoch 8/17
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 0.0300 - val_loss: 0.0216
Epoch 9/17
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 0.0287 - val_loss: 0.0188
Epoch 10/17
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 68ms/step - loss: 0.0307 - val_loss: 0.0187
Epoch 11/17
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 70ms/step - loss: 0.0267 - val_loss: 0.0205
Epoch 12/17
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 95ms/step - loss: 0.

In [ ]:
# y_pred_adv0 = model2.predict(x_valid0)
# y_pred1= model2.predict(x1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


In [ ]:
# mean_absolute_error0 = mse(y_pred_adv0, y_valid0)
# mean_absolute_error1= mse(y_pred1, y1)

In [ ]:
# mean_absolute_error0

<tf.Tensor: shape=(), dtype=float32, numpy=0.004520685411989689>

In [ ]:
# mean_absolute_error1

<tf.Tensor: shape=(), dtype=float32, numpy=0.15394073724746704>